Doing the calculation to correct the results given in the [glmmTMB covstruct vignette](https://glmmtmb.github.io/glmmTMB/articles/covstruct.html):

> For an unstructured matrix of size `n`, parameters `1:n` represent the log-standard deviations while the remaining `n(n-1)/2` (i.e. `(n+1):(n:(n*(n+1)/2))`) are the elements of the scaled Cholesky factor of the correlation matrix, filled in row-wise order (see TMB documentation). In particular, if L is the lower-triangular matrix with 1 on the diagonal and the correlation parameters in the lower triangle, then the correlation matrix is defined as $\Sigma = D^{-1/2} L L^\top D^{-1/2}$, where $D=\textrm{diag}(LL^\top)$. For a single correlation parameter $\theta_0$, this works out to $\rho=\theta_0/\sqrt(1+\theta^2_0)$.


In [1]:
from sympy import *
import numpy as np
import IPython.display as disp  ## for pretty display

## references/figuring out how to do stuff:
## https://stackoverflow.com/questions/20979993/how-to-pretty-print-in-ipython-notebook-via-sympy
## https://docs.sympy.org/latest/tutorial/matrices.html
## https://stackoverflow.com/questions/33104953/how-to-replace-the-the-diagonal-elements-of-a-matrix-by-a-vector-in-sympy
## https://docs.sympy.org/latest/modules/tensor/indexed.html

In [2]:
theta = Symbol("theta")
m = Matrix([[1,0],[theta,1]])
def corcalc(m):
    n = m.shape[0]
    S = m*m.T
    d = np.diag(S)
    Dinv = Matrix(n, n, lambda i,j: 1/sqrt(S[i,i]) if i==j else 0)
    res = Dinv*S*Dinv
    return res

disp.display(corcalc(m))

Matrix([
[                       1, theta/sqrt(theta**2 + 1)],
[theta/sqrt(theta**2 + 1),                        1]])

Now generalize to $n>2$: first by brute force ...

In [3]:
theta = IndexedBase('theta')
m = Matrix([[1,0,0], [theta[0], 1, 0], [theta[1], theta[2], 1]] )
disp.display(corcalc(m))

Matrix([
[                                           1,                                                             theta[0]/sqrt(theta[0]**2 + 1),                                               theta[1]/sqrt(theta[1]**2 + theta[2]**2 + 1)],
[              theta[0]/sqrt(theta[0]**2 + 1),                                                                                          1, (theta[0]*theta[1] + theta[2])/(sqrt(theta[0]**2 + 1)*sqrt(theta[1]**2 + theta[2]**2 + 1))],
[theta[1]/sqrt(theta[1]**2 + theta[2]**2 + 1), (theta[0]*theta[1] + theta[2])/(sqrt(theta[0]**2 + 1)*sqrt(theta[1]**2 + theta[2]**2 + 1)),                                                                                          1]])

Now define a general function for "lower triangular matrix indexed by $\theta$"

In [6]:
def ltrimat(n):
    k = [0] ## mutable
    def nextval(i,j):
        if i>j:
            r = theta[k[0]]
            k[0] = k[0]+1
            return r
        elif i==j:
            return 1
        return 0
    m = Matrix(n, n, lambda i,j: nextval(i,j))
    return(m)


Matrix([
[       1,        0,        0, 0],
[theta[0],        1,        0, 0],
[theta[1], theta[2],        1, 0],
[theta[3], theta[4], theta[5], 1]])

Check that we're indeed filling in elements as specified in the definition:

In [8]:
disp.display(ltrimat(4))

Matrix([
[       1,        0,        0, 0],
[theta[0],        1,        0, 0],
[theta[1], theta[2],        1, 0],
[theta[3], theta[4], theta[5], 1]])

Now try `corcalc()` for some larger examples:

In [7]:
disp.display(corcalc(ltrimat(4)))

Matrix([
[                                                         1,                                                                           theta[0]/sqrt(theta[0]**2 + 1),                                                                                               theta[1]/sqrt(theta[1]**2 + theta[2]**2 + 1),                                                                                 theta[3]/sqrt(theta[3]**2 + theta[4]**2 + theta[5]**2 + 1)],
[                            theta[0]/sqrt(theta[0]**2 + 1),                                                                                                        1,                                                 (theta[0]*theta[1] + theta[2])/(sqrt(theta[0]**2 + 1)*sqrt(theta[1]**2 + theta[2]**2 + 1)),                                   (theta[0]*theta[3] + theta[4])/(sqrt(theta[0]**2 + 1)*sqrt(theta[3]**2 + theta[4]**2 + theta[5]**2 + 1))],
[              theta[1]/sqrt(theta[1]**2 + theta[2]**2 + 1),               (theta[0]*theta[1] +

In [5]:
disp.display(corcalc(ltrimat(5)))

Matrix([
[                                                                       1,                                                                                         theta[0]/sqrt(theta[0]**2 + 1),                                                                                                             theta[1]/sqrt(theta[1]**2 + theta[2]**2 + 1),                                                                                                                                 theta[3]/sqrt(theta[3]**2 + theta[4]**2 + theta[5]**2 + 1),                                                                                                                   theta[6]/sqrt(theta[6]**2 + theta[7]**2 + theta[8]**2 + theta[9]**2 + 1)],
[                                          theta[0]/sqrt(theta[0]**2 + 1),                                                                                                                      1,                                                               (theta[0